### https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

## Not fully working. Work later.

In [56]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(seed)

Music 21 is a Python library developed by MIT for understanding music data. MIDI is a standard format for storing music files (it stands for Musical Instrument Digital Interface). MIDI files contain the instructions rather than the actual audio. Hence, it occupies very little memory. That’s why it is usually preferred while transferring files.

In [57]:
#!pip install music21

In [58]:
#dealing with midi files
from music21 import * 

#array processing
import numpy as np     
import os

#random number generator
import random         

#keras for building deep learning model
from keras.layers import * 
from keras.models import *
import keras.backend as K

### Reading data:

In [59]:
def read_midi(file):
  notes=[]
  notes_to_parse = None

  #parsing a midi file
  midi = converter.parse(file)
  #grouping based on different instruments
  s2 = instrument.partitionByInstrument(midi)

  #Looping over all the instruments
  for part in s2.parts:
    #select elements of only piano
    if 'Piano' in str(part): 
      notes_to_parse = part.recurse() 
      #finding whether a particular element is note or a chord
      for element in notes_to_parse:
        if isinstance(element, note.Note):
          notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))
      
  return notes

### Read the MIDI files from the directory:

In [60]:
#read all the filenames
files=[i for i in os.listdir() if i.endswith(".mid")]

#reading each midi file
all_notes=[]
for i in files:
  all_notes.append(read_midi(i))

#notes and chords of all the midi files
notes = [element for notes in all_notes for element in notes]

In [61]:
notes

['F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F2',
 'F#3',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 'F#3',
 'F2',
 '

In [62]:
# Preparing the input and output sequences as mentioned in the article:

In [63]:
#length of a input sequence
no_of_timesteps = 128      

#no. of unique notes
n_vocab = len(set(notes))  

#all the unique notes
pitch = sorted(set(item for item in notes))  

#assign unique value to every note
note_to_int = dict((note, number) for number, note in enumerate(pitch))  

#preparing input and output sequences
X = []
y = []
for notes in all_notes:
  for i in range(0, len(notes) - no_of_timesteps, 1):
    input_ = notes[i:i + no_of_timesteps]
    output = notes[i + no_of_timesteps]
    X.append([note_to_int[note] for note in input_])
    y.append(note_to_int[output])

In [64]:
#reshaping
X = np.reshape(X, (len(X), no_of_timesteps, 1))
#normalizing the inputs
X = X / float(n_vocab)  

In [65]:
def lstm():
  model = Sequential()
  model.add(LSTM(128,return_sequences=True))
  model.add(LSTM(128))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

In [66]:
K.clear_session()
def simple_wavenet():
  no_of_kernels=64
  num_of_blocks= int(np.sqrt(no_of_timesteps)) - 1   #no. of stacked conv1d layers

  model = Sequential()
  for i in range(num_of_blocks):
    model.add(Conv1D(no_of_kernels,3,dilation_rate=(2**i),padding='causal',activation='relu'))
  model.add(Conv1D(1, 1, activation='relu', padding='causal'))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(n_vocab, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

In [67]:
import keras
mc = keras.callbacks.ModelCheckpoint('model{epoch:03d}.h5', save_weights_only=False, period=50)

In [68]:
print("Eager execution: {}".format(tf.executing_eagerly()))

Eager execution: True


In [69]:
#inputs = keras.Input(shape=(None, 3))  # variable number of timesteps each with length 3
#inputs = keras.Input(shape=(4, 3))     # 4 timesteps each with length 3
#inputs = keras.Input(shape=(4, None))  # this is WRONG! you can't do this. Number of features must be fixed

#label = keras.layers.SimpleRNN(units=5, activation='softmax')(inputs)

#inputs = keras.Input(shape=(3,))
#embedding = Embedding(voc_size, embed_dim, ..)
#X = embedding(inputs)

In [70]:
#inputs = keras.Input(shape=(3,))
#embedding = Embedding(X.shape[0], X.shape[1:])
#X = embedding(inputs)

In [71]:
#embedding

In [72]:
#print(X.shape, np.array(y).shape)

In [73]:
#data = np.array([[1, 2, 3], [3, 4, 5]])
#labels = np.array([1, 2])
#print (data.shape, labels.shape)

In [74]:
import keras
mc = keras.callbacks.ModelCheckpoint('model{epoch:03d}.h5', save_weights_only=False, period=50)

In [75]:
model = simple_wavenet()
model.fit(X,np.array(y), epochs=300, batch_size=128,callbacks=[mc])

TypeError: unsupported operand type(s) for %: 'builtin_function_or_method' and 'int'

In [76]:
def generate_music(model, pitch, no_of_timesteps, pattern):
    
    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    prediction_output = []
    
    # generate 50 elements
    for note_index in range(50):
        #reshaping array to feed into model
        input_ = np.reshape(pattern, (1, len(pattern), 1))
        #predict the probability and choose the maximum value
        proba = model.predict(input_, verbose=0)
        index = np.argmax(proba)
        #convert integer back to the element
        pred = int_to_note[index]
        prediction_output.append(pred)
        pattern = list(pattern)
        pattern.append(index/float(n_vocab))
        #leave the first value at index 0
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [77]:
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Specify duration between 2 notes
        offset += 0.5
        # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [78]:
#Select random chunk for the first iteration
start = np.random.randint(0, len(X)-1)
pattern = X[start]
#load the best model
model=load_model('model300.h5')
#generate and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
convert_to_midi(music)

OSError: Unable to open file (unable to open file: name = 'model300.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)